# 01 - Operaciones CRUD en MongoDB

Este notebook demuestra las operaciones básicas **CRUD** (Create, Read, Update, Delete) sobre la base de datos de Airbnb Madrid.

## Contenido
1. Conexión a MongoDB
2. Lectura de datos (Read)
3. Consultas básicas
4. Búsqueda por campos específicos
5. Actualización de documentos (Update)
6. Eliminación de documentos (Delete)
7. Estadísticas de la colección

In [1]:
# Imports necesarios
import sys
from pathlib import Path
sys.path.insert(0, str(Path.cwd().parent))

from src.crud_operations import AirbnbCRUD
from src.database import MongoDBConnection
import pandas as pd
from pprint import pprint

print("✅ Imports completados")

✅ Imports completados


## 1. Conexión a MongoDB

In [2]:
# Conectar a MongoDB
conn = MongoDBConnection()
crud = AirbnbCRUD()

# Verificar conexión
if conn.ping():
    print("✅ Conexión exitosa a MongoDB")
    total = crud.get_total_listings()
    print(f"📊 Total de listings: {total:,}")
else:
    print("❌ Error de conexión")

✅ Conexión exitosa a MongoDB
📊 Total de listings: 26,004


## 2. Lectura de Datos (Read)

### 2.1 Obtener un listing por ID

In [3]:
# Obtener el primer listing para ver su estructura
first_listing = crud.collection.find_one()

if first_listing:
    print("📋 Ejemplo de Listing:")
    print(f"\nID: {first_listing.get('id', first_listing.get('_id'))}")
    print(f"Nombre: {first_listing.get('name')}")
    print(f"Barrio: {first_listing.get('neighbourhood_cleansed', 'N/A')}")
    print(f"Precio: {first_listing.get('price')}€")
    print(f"Tipo: {first_listing.get('room_type')}")
    print(f"Reviews: {first_listing.get('number_of_reviews', 0)}")
    
    # Ver todos los campos disponibles
    print(f"\n📝 Campos disponibles ({len(first_listing)} campos):")
    print(list(first_listing.keys())[:20], "...")

📋 Ejemplo de Listing:

ID: 21853
Nombre: Bright and airy room
Barrio: Cármenes
Precio: 29.0€
Tipo: Private room
Reviews: 33

📝 Campos disponibles (82 campos):
['_id', 'id', 'listing_url', 'scrape_id', 'last_scraped', 'source', 'name', 'description', 'neighborhood_overview', 'picture_url', 'host_id', 'host_url', 'host_name', 'host_since', 'host_location', 'host_about', 'host_response_time', 'host_response_rate', 'host_acceptance_rate', 'host_is_superhost'] ...


### 2.2 Obtener múltiples listings

In [4]:
# Obtener los primeros 10 listings
listings = list(crud.collection.find().limit(10))

print(f"📊 Primeros 10 listings:\n")
for i, listing in enumerate(listings, 1):
    print(f"{i}. {listing.get('name')[:50]}... - {listing.get('price')}€ - {listing.get('neighbourhood_cleansed', 'N/A')}")

📊 Primeros 10 listings:

1. Bright and airy room... - 29.0€ - Cármenes
2. Apartamentos Dana Sol... - 0.0€ - Sol
3. Beautiful loft in Madrid Center... - 0.0€ - Embajadores
4. Apartasol Apartamentos Dana... - 0.0€ - Universidad
5. MAGIC ARTISTIC HOUSE IN THE CENTER OF MADRID... - 64.0€ - Justicia
6. Tu hogar en centro de Madrid.... - 86.0€ - Universidad
7. Sunny attic duplex flat with terrace next to Sol... - 196.0€ - Embajadores
8. Retiro Park, Stay at cosy studio... - 0.0€ - Recoletos
9. Cool Apart. (10min Center + WIFI)... - 72.0€ - Embajadores
10. Cozy attic with intimate rooftop terrace+ elevator... - 209.0€ - Embajadores


## 3. Consultas Básicas

### 3.1 Filtrar por barrio

In [5]:
# Buscar listings en un barrio específico
barrio = "Centro"  # Cambia esto por un barrio de tu dataset

# Primero, ver qué barrios hay
barrios = crud.collection.distinct('neighbourhood_cleansed')
print(f"📍 Barrios disponibles ({len(barrios)}): {barrios[:10]}...\n")

# Buscar en el primer barrio
if barrios:
    barrio = barrios[0]
    listings_barrio = list(crud.collection.find({'neighbourhood_cleansed': barrio}).limit(5))
    
    print(f"🏘️ Listings en {barrio}:")
    for listing in listings_barrio:
        print(f"  • {listing.get('name')[:40]}... - {listing.get('price')}€")

📍 Barrios disponibles (128): ['Abrantes', 'Acacias', 'Adelfas', 'Aeropuerto', 'Aguilas', 'Alameda de Osuna', 'Almagro', 'Almenara', 'Almendrales', 'Aluche']...

🏘️ Listings en Abrantes:
  • Lovely flat In Madrid... - 85.0€
  • Accommodation Madrid, nice apartment... - 20.0€
  • Habitacion luminosa y tranquila en Carab... - 0.0€
  • MAGNÍFICO PISO FINAL CHAMPIONS LIGUE. WI... - 0.0€
  • habitación privada en abrantes... - 100.0€


### 3.2 Filtrar por rango de precio

In [6]:
# Buscar listings entre 50€ y 100€
precio_min = 50
precio_max = 100

listings_precio = list(crud.collection.find({
    'price': {'$gte': precio_min, '$lte': precio_max}
}).limit(10))

print(f"💰 Listings entre {precio_min}€ y {precio_max}€:")
for listing in listings_precio:
    print(f"  • {listing.get('name')[:40]}... - {listing.get('price')}€ - {listing.get('room_type')}")

💰 Listings entre 50€ y 100€:
  • MAGIC ARTISTIC HOUSE IN THE CENTER OF MA... - 64.0€ - Private room
  • Tu hogar en centro de Madrid.... - 86.0€ - Entire home/apt
  • Cool Apart. (10min Center + WIFI)... - 72.0€ - Entire home/apt
  • Atocha Attic - Unbeatable Location... - 98.0€ - Entire home/apt
  • Central and quiet.... - 94.0€ - Entire home/apt
  • NICE CENTRAL TOURISM-BUSINESS APTO... - 87.0€ - Entire home/apt
  • alquilo habitación... - 75.0€ - Private room
  • private house B & B. Arturo Soria (Metro... - 90.0€ - Private room
  • Habitación en ático muy céntrico con dos... - 70.0€ - Private room
  • Apartment in the heart of Madrid... - 98.0€ - Entire home/apt


### 3.3 Buscar por tipo de habitación

In [7]:
# Ver tipos de habitación disponibles
room_types = crud.collection.distinct('room_type')
print(f"🏠 Tipos de habitación: {room_types}\n")

# Contar por tipo
total_listings = crud.collection.count_documents({})
for room_type in room_types:
    count = crud.collection.count_documents({'room_type': room_type})
    print(f"  {room_type}: {count:,} ({count/total_listings*100:.1f}%)")

🏠 Tipos de habitación: ['Entire home/apt', 'Hotel room', 'Private room', 'Shared room']

  Entire home/apt: 16,959 (65.2%)
  Hotel room: 81 (0.3%)
  Private room: 8,806 (33.9%)
  Shared room: 158 (0.6%)


## 4. Consultas Avanzadas

### 4.1 Búsqueda por múltiples criterios

In [8]:
# Buscar: vivienda completa, precio < 100€, mínimo 10 reviews
query = {
    'room_type': 'Entire home/apt',
    'price': {'$lt': 100},
    'number_of_reviews': {'$gte': 10}
}

results = list(crud.collection.find(query).limit(10))

print(f"🔍 Encontrados: {len(results)} listings")
print(f"\n✅ Viviendas completas < 100€ con 10+ reviews:\n")
for listing in results:
    print(f"  • {listing.get('name')[:40]}...")
    print(f"    Precio: {listing.get('price')}€ | Reviews: {listing.get('number_of_reviews')} | Barrio: {listing.get('neighbourhood_cleansed', 'N/A')}")
    print()

🔍 Encontrados: 10 listings

✅ Viviendas completas < 100€ con 10+ reviews:

  • Apartamentos Dana Sol...
    Precio: 0.0€ | Reviews: 172 | Barrio: Sol

  • Apartasol Apartamentos Dana...
    Precio: 0.0€ | Reviews: 49 | Barrio: Universidad

  • Tu hogar en centro de Madrid....
    Precio: 86.0€ | Reviews: 63 | Barrio: Universidad

  • Retiro Park, Stay at cosy studio...
    Precio: 0.0€ | Reviews: 105 | Barrio: Recoletos

  • Cool Apart. (10min Center + WIFI)...
    Precio: 72.0€ | Reviews: 184 | Barrio: Embajadores

  • Atocha Attic - Unbeatable Location...
    Precio: 98.0€ | Reviews: 112 | Barrio: Palos de Moguer

  • Central and quiet....
    Precio: 94.0€ | Reviews: 711 | Barrio: Palacio

  • DUPLEX AT MADRID CENTER WITH POOL...
    Precio: 0.0€ | Reviews: 25 | Barrio: Acacias

  • NICE CENTRAL TOURISM-BUSINESS APTO...
    Precio: 87.0€ | Reviews: 189 | Barrio: Rios Rosas

  • Apartment in the heart of Madrid...
    Precio: 98.0€ | Reviews: 844 | Barrio: Sol



### 4.2 Top 10 más baratos

In [9]:
# Top 10 más baratos
cheapest = list(crud.collection.find({'price': {'$gt': 0}}).sort('price', 1).limit(10))

print("💵 Top 10 más baratos:\n")
for i, listing in enumerate(cheapest, 1):
    print(f"{i}. {listing.get('name')[:40]}... - {listing.get('price')}€")

💵 Top 10 más baratos:

1. Habitación cómoda en Moratalaz... - 1.0€
2. Habitación cómoda, alameda Osuna... - 7.0€
3. Piso en Salamanca, centro... - 7.0€
4. Apartamento En Madrid... - 8.0€
5. Centrica Habitación... - 8.0€
6. bonita habitación para dama... - 8.0€
7. Casa Chako  Mi gato en la ventana... - 8.0€
8. Excelente habitación... - 8.0€
9. Un espacio agradable y cómodo.... - 9.0€
10. Bello alojamiento en manzanares... - 9.0€


### 4.3 Top 10 más caros

In [10]:
# Top 10 más caros
expensive = list(crud.collection.find().sort('price', -1).limit(10))

print("💎 Top 10 más caros:\n")
for i, listing in enumerate(expensive, 1):
    print(f"{i}. {listing.get('name')[:40]}... - {listing.get('price')}€")

💎 Top 10 más caros:

1. Madrid... - 32000.0€
2. Captivating Apartment in Madrid near El ... - 21000.0€
3. Piso en goya... - 20540.0€
4. Apartamento... - 19000.0€
5. Bohemian Charm in the Latina.... - 18600.0€
6. Plaza Mayor/apartamento Latina... - 15051.0€
7. Apart jardines San Vicente... - 10000.0€
8. Touristenwohnung Santa Ana mit Balkon... - 9999.0€
9. 3 Pièces 6 Personnes 623384... - 9999.0€
10. 2 pièces 2 personnes Standard 165094... - 9999.0€


## 5. Estadísticas de la Colección

In [11]:
# Estadísticas generales
stats = conn.get_collection_stats()

print("📊 ESTADÍSTICAS DE LA COLECCIÓN:\n")
print(f"  • Total documentos: {stats['count']:,}")
print(f"  • Tamaño: {stats['size'] / 1024 / 1024:.2f} MB")
print(f"  • Tamaño promedio por documento: {stats['avgObjSize'] / 1024:.2f} KB")
print(f"  • Índices: {stats['indexes']}")
print(f"  • Tamaño de índices: {stats['indexSize'] / 1024 / 1024:.2f} MB")

📊 ESTADÍSTICAS DE LA COLECCIÓN:

  • Total documentos: 26,004
  • Tamaño: 93.54 MB
  • Tamaño promedio por documento: 3.68 KB
  • Índices: 1
  • Tamaño de índices: 0.27 MB


## 6. Agregaciones Simples

### 6.1 Precio promedio por barrio

In [12]:
# Precio promedio por barrio (top 10)
pipeline = [
    {'$match': {'price': {'$gt': 0}}},
    {'$group': {
        '_id': '$neighbourhood_cleansed',
        'precio_promedio': {'$avg': '$price'},
        'total_listings': {'$sum': 1}
    }},
    {'$sort': {'precio_promedio': -1}},
    {'$limit': 10}
]

results = list(crud.collection.aggregate(pipeline))

print("💰 Top 10 barrios más caros (precio promedio):\n")
for i, result in enumerate(results, 1):
    barrio = result['_id'] or 'Sin barrio'
    precio = result['precio_promedio']
    total = result['total_listings']
    print(f"{i}. {barrio}: {precio:.2f}€ promedio ({total} listings)")

💰 Top 10 barrios más caros (precio promedio):

1. Palomeras Bajas: 447.45€ promedio (55 listings)
2. Fuentelareina: 347.20€ promedio (5 listings)
3. Piovera: 314.41€ promedio (27 listings)
4. Castillejos: 281.98€ promedio (253 listings)
5. Valdefuentes: 246.72€ promedio (126 listings)
6. Hellín: 246.33€ promedio (15 listings)
7. Ibiza: 236.88€ promedio (225 listings)
8. Castellana: 225.03€ promedio (155 listings)
9. Recoletos: 222.13€ promedio (271 listings)
10. Jerónimos: 215.91€ promedio (89 listings)


### 6.2 Distribución por tipo de habitación

In [13]:
# Distribución por tipo de habitación
pipeline = [
    {'$group': {
        '_id': '$room_type',
        'count': {'$sum': 1},
        'precio_promedio': {'$avg': '$price'}
    }},
    {'$sort': {'count': -1}}
]

results = list(crud.collection.aggregate(pipeline))
total = crud.get_total_listings()

print("🏠 Distribución por tipo de habitación:\n")
for result in results:
    tipo = result['_id']
    count = result['count']
    precio = result['precio_promedio']
    pct = count / total * 100
    print(f"  {tipo}:")
    print(f"    • Cantidad: {count:,} ({pct:.1f}%)")
    print(f"    • Precio promedio: {precio:.2f}€")
    print()

🏠 Distribución por tipo de habitación:

  Entire home/apt:
    • Cantidad: 16,959 (65.2%)
    • Precio promedio: 144.55€

  Private room:
    • Cantidad: 8,806 (33.9%)
    • Precio promedio: 50.64€

  Shared room:
    • Cantidad: 158 (0.6%)
    • Precio promedio: 45.87€

  Hotel room:
    • Cantidad: 81 (0.3%)
    • Precio promedio: 148.72€



## 7. Conversión a Pandas DataFrame

In [ ]:
# Obtener datos y convertir a DataFrame
listings = list(crud.collection.find().limit(1000))
df = pd.DataFrame(listings)

print(f"📊 DataFrame creado: {df.shape[0]} filas x {df.shape[1]} columnas\n")

# Seleccionar columnas importantes
cols_importantes = ['name', 'price', 'neighbourhood_cleansed', 'room_type', 'number_of_reviews']
cols_disponibles = [col for col in cols_importantes if col in df.columns]

if cols_disponibles:
    df_simple = df[cols_disponibles]
    print("Vista previa del DataFrame:")
    display(df_simple.head(10))
    
    # Estadísticas
    if 'price' in df.columns:
        print(f"\n💰 Estadísticas de precio:")
        print(df['price'].describe())

## 🎉 Resumen

En este notebook aprendiste:

- ✅ Conectar a MongoDB
- ✅ Operaciones CRUD básicas
- ✅ Consultas y filtros
- ✅ Agregaciones simples
- ✅ Conversión a Pandas

**Siguiente:** `02_data_analysis.ipynb` - Análisis exploratorio de datos